# Calculate

In [2]:
from DMDana.lib.logic_func import *
from DMDana.lib.parallel_folder_analysis import *
from DMDana.lib.pandas_figure_gen import *

In [ ]:
logic_func=analyze
df=parallel_folder_analysis(analysis_functions=logic_func.func_list,core_num=20,index_shift=2,file_prefix=logic_func.file_prefix)

In [ ]:
logic_func=summary
df=parallel_folder_analysis(analysis_functions=logic_func.func_list,core_num=20,index_shift=2,file_prefix=logic_func.file_prefix)

# Visualization

In [ ]:
from DMDana.lib.logic_func import *
from DMDana.lib.parallel_folder_analysis import *
from DMDana.lib.pandas_figure_gen import *


In [ ]:
EXCEL_INDEX_SHIFT=2
data=pd.read_excel('summary_database_out.xlsx')
data['excel_index']=data.index+EXCEL_INDEX_SHIFT
data=data.set_index('excel_index')
for i in data.columns:
    print(i)

In [ ]:
with open('Group') as f:
    content=f.readlines()
    for i,row in data.iterrows():
        data.loc[row.name,'Group']=content[row.name-EXCEL_INDEX_SHIFT].strip()
with open('System') as f:
    content=f.readlines()
    for i,row in data.iterrows():
        data.loc[row.name,'System']=content[row.name-EXCEL_INDEX_SHIFT].strip()

part_col=['System','Group','Light_pumpE','Light_pumpPoltype','Light_pumpA0','impurity_density','DM_upper_bound','DM_lower_bound','mu_eV','Kpoint-num','NkMult','Recomb_tau','RTA_tau_ps','bStart_tau','bEnd_tau','Eph-ePhDelta','EE-epsilon','DC_x','DC_y',"DC_z",'DC_diag_x','DC_diag_y',"DC_diag_z",'DC_offdiag_x','DC_offdiag_y',"DC_offdiag_z",'Cutoff_time_for_current_and_spectrum_calculation',"Max_Occupation_change_Conduction",'total_time_fs','Max_Occupation_change_Valence','mu_Boltz(eV)','T_Boltz(K)','time_fs_to_evaluate_occupation_change','folder']

part=data[part_col]
part.to_excel('ana.xlsx',index=False)

In [ ]:
pd.set_option('display.max_columns', None)
gaas_data=data[data['System'].str.contains('GaAs')]


In [ ]:
gaas_data['DC_diag_x_percent']=gaas_data['DC_diag_x']/gaas_data['DC_x']
gaas_data['DC_diag_y_percent']=gaas_data['DC_diag_y']/gaas_data['DC_y']
gaas_data['DC_diag_z_percent']=gaas_data['DC_diag_z']/gaas_data['DC_z']
gaas_data['DC_offdiag_x_percent']=gaas_data['DC_offdiag_x']/gaas_data['DC_x']
gaas_data['DC_offdiag_y_percent']=gaas_data['DC_offdiag_y']/gaas_data['DC_y']
gaas_data['DC_offdiag_z_percent']=gaas_data['DC_offdiag_z']/gaas_data['DC_z']

In [ ]:
plot_multi_y_axis(gaas_data.sort_values(by='Max_Occupation_change_Conduction',ascending=False), 'System', ['Max_Occupation_change_Conduction','Max_Occupation_change_Valence'])
plot_multi_y_axis(gaas_data.sort_values(by='Max_Occupation_change_Conduction',ascending=False), 'System', ['T_Boltz(K)','mu_Boltz(eV)'])
plot_multi_y_axis(gaas_data.sort_values(by='Max_Occupation_change_Conduction',ascending=False), 'System', ['Max_Occupation_change_Conduction','Max_Occupation_change_Valence']+['T_Boltz(K)','mu_Boltz(eV)'])
plot_multi_y_axis(gaas_data.sort_values(by='DC_z',ascending=True), 'System', ['DC_x','DC_z','DC_y'])
plot_multi_y_axis(gaas_data.sort_values(by='DC_z',ascending=True), 'System',['DC_x','DC_diag_x','DC_offdiag_x'],sameaxis=True)
plot_multi_y_axis(gaas_data.sort_values(by='DC_z',ascending=True), 'System',['DC_y','DC_diag_y','DC_offdiag_y'],sameaxis=True)
plot_multi_y_axis(gaas_data.sort_values(by='DC_z',ascending=True), 'System',['DC_z','DC_diag_z','DC_offdiag_z'],sameaxis=True)
#plot_multi_y_axis(gaas_data.sort_values(by='DC_z',ascending=True), 'System',['DC_offdiag_x_percent','DC_offdiag_y_percent','DC_offdiag_z_percent'])

In [ ]:
             
patterns = {
    'conduction_occup_image_path': '*heatmap_Emin1.420eV_Emax1.610eV_fitted*',
    'valence_occup_image_path': '*_heatmap_Emin-0.131eV_Emax0.010eV*',
}

annotations = [
    (0, "mu_Boltz(eV)={mu_Boltz(eV):.3f}\nT_Boltz(K)={T_Boltz(K):.1f}\nFinal_Max_Change={Max_Occupation_change_Conduction:.3f}", 
     ((0.8, 0.8), 'right', 'top')),  
    (1, "Final_Max_Change={Max_Occupation_change_Valence:.3f}", 
     ((0.2, 0.2), 'left', 'bottom'))
]
title="No.{name} {System}"
display_images(gaas_data.sort_values(by='Max_Occupation_change_Conduction',ascending=False), patterns, annotations,title=title)





In [ ]:
current_annotations = []
directions = ['x', 'y', 'z']
components = ['', '_diag', '_offdiag']
for dir_i, dir in enumerate(directions):
    for compo_i, compo in enumerate(components):
        annotation_text = f"Final_DC{compo}_{dir}={{DC{compo}_{dir}:.2e}}"
        position = ((compo_i * 2 + 1) / 6, (dir_i * 2 + 1) / 6)  # 计算位置
        current_annotations.append((0, annotation_text, (position, 'center', 'center')))  
current_patterns = {
    'current_image_path': '*j_smooth_off*',
    'FFT_linear_image_path': '*Window_type=Flattop;log_yFalse-j-fft*',
    'FFT_log_image_path': '*Window_type=Flattop;log_yTrue-j-fft*'
    
}
title="No.{name} {System}"
display_images(gaas_data.sort_values(by='DC_z',ascending=False), current_patterns, current_annotations,title=title)

In [ ]:
plot_correlation_heatmap(gaas_data, ['T_Boltz(K)', 'mu_Boltz(eV)', 'Max_Occupation_change_Conduction', 'Max_Occupation_change_Valence','mu_eV'], 'GaAs')